In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO


In [ ]:
from selenium import webdriver
import time
driver = webdriver.Chrome() 
driver.get('https://www.safekorea.go.kr/idsiSFK/neo/sfk/cs/sfc/dis/disasterMsgList.jsp?menuSeq=679')
time.sleep(1)

input_elem = driver.find_element(By.ID, "datepicker1")
input_elem.clear()
input_elem.send_keys("2025-07-23")
time.sleep(1)

input_elem = driver.find_element(By.ID, "datepicker2")
input_elem.clear()
input_elem.send_keys("2025-07-24")
time.sleep(1)

search_button = driver.find_element(By.CLASS_NAME, "search_btn")
search_button.click()
time.sleep(1)

from bs4 import BeautifulSoup
from io import StringIO
max_element =  driver.find_element(By.ID, "tbpagetotal")
max_text = max_element.text[1:]

dfs = []
for page_num in range(int(max_text)):
    table_element = driver.find_element(By.CLASS_NAME, "boardList_table")
    html = table_element.get_attribute("outerHTML")

    soup = BeautifulSoup(html, "html.parser")
    html_str = str(soup)

    df = pd.read_html(StringIO(html_str))[0]

    dfs.append(df)
    
    search_button = driver.find_element(By.ID, "apagenext")
    search_button.click()
    time.sleep(1)
df_all = pd.concat(dfs)

In [32]:
df_all = pd.concat(dfs)

In [4]:
from selenium import webdriver
import time
driver = webdriver.Chrome() 
driver.get('https://www.safekorea.go.kr/idsiSFK/neo/sfk/cs/sfc/dis/disasterMsgView.jsp?menuSeq=679&md101_sn=242881')
time.sleep(1)




In [11]:
from selenium.webdriver.common.by import By
date_elem = driver.find_element(By.ID, "smsTitle")
region_elem = driver.find_element(By.ID, "bbsDetail_0_cdate")

print(date_elem.text[0:19])
print(region_elem.text)

2025/07/26 14:11:23
강원특별자치도 양구군


In [39]:
 df_all['번호'].tolist()[0:10]

[242523,
 242522,
 242521,
 242520,
 242519,
 242518,
 242517,
 242516,
 242515,
 242514]

In [43]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import pandas as pd

driver = webdriver.Chrome()
nums = []
dates = []
regions = [] 
msgs = []
for md101_sn in df_all['번호'].tolist()[0:10]:
    driver.get(f'https://www.safekorea.go.kr/idsiSFK/neo/sfk/cs/sfc/dis/disasterMsgView.jsp?menuSeq=679&md101_sn={md101_sn}')
    time.sleep(0.5)
    date_elem = driver.find_element(By.ID, "smsTitle")
    region_elem = driver.find_element(By.ID, "bbsDetail_0_cdate")
    msg_elem = driver.find_element(By.ID, "msg_cn")
    region_elem_split = [item.strip() for item in region_elem.text.split(',')]
    for region in region_elem_split:
        nums.append(md101_sn)
        dates.append(date_elem.text[0:19])
        regions.append(region)
        msgs.append(msg_elem.text)

df_webpage = pd.DataFrame({
    "번호": nums,
    "date": dates,
    "region": regions,
    "message": msgs
})


In [48]:
df_merged = pd.merge(df_webpage, df_all, on='번호', how='left')[['번호', 'date', '재해구분', 'region', 'message']]
df_merged

,번호,date,재해구분,region,message
0,242523,2025/07/24 21:36:02,화재,경기도 안성시 서운면,"오늘 20:30분경, 서운면 신촌리 돼지농장에서 화재발생. 서운로를 통행하시는 운전..."
1,242522,2025/07/24 20:54:01,화재,충청남도 아산시,오늘 20:23 배방읍 공수리 63-17 건물에서 화재 발생. 차량은 주변 도로로 ...
2,242521,2025/07/24 20:02:44,호우,경상남도 산청군 단성면,"(단수안내) 7/24(목)22시~7/25(금)06시 단성,신안,생비량,신등면 전역 ..."
3,242521,2025/07/24 20:02:44,호우,경상남도 산청군 신안면,"(단수안내) 7/24(목)22시~7/25(금)06시 단성,신안,생비량,신등면 전역 ..."
4,242520,2025/07/24 17:58:27,기타,부산광역시 강서구,"강서구 송정동 성고개(용원에서 녹산 방향)사면 일부 유실에 따른 정비·진단 중으로,..."
5,242519,2025/07/24 17:42:06,기타,전라남도 목포시,7.25.(금)~27.(일)(오전01~05시) 바닷물 수위가 최고 5m이상 상승이 ...
6,242518,2025/07/24 17:30:25,붕괴,경기도 연천군,"호우 및 댐방류로 인한 한탄강/임진강의 수위가 높으니 계곡, 하천, 하천산책로, 침..."
7,242517,2025/07/24 17:22:56,호우,경상남도 산청군 단성면,"(단수안내) 7/24(목)22시~7/25(금)06시 단성,신안,생비량,신등면 전역 ..."
8,242517,2025/07/24 17:22:56,호우,경상남도 산청군 신안면,"(단수안내) 7/24(목)22시~7/25(금)06시 단성,신안,생비량,신등면 전역 ..."
9,242516,2025/07/24 17:21:33,폭염,경상남도 남해군,"남해군에 연일 더위가 지속되고 있습니다▲외출 자제▲야외작업(논밭작업, 공사현장 등)..."
